In [1]:
from driftdb.drift_evaluator import run_new_data_evaluator, DetectOutlierNewDataEvaluator
from dotenv import load_dotenv
from github import Github
import os




load_dotenv("../../.env")
gh_token = os.getenv("GH_TOKEN")
gh_client = Github(gh_token)
repo = os.getenv("REPO") or "gh_org/repo"
commit_sha = "8e1edb9b7b518aed0529a973d14cc7f8e4532529"
drift_evaluator = DetectOutlierNewDataEvaluator(numerical_cols=["age"], categorical_cols=[])

drift = run_new_data_evaluator(gh_client=gh_client, repo_name=repo, commit_sha=commit_sha, drift_evaluator=drift_evaluator)
drift

In [ ]:
from driftdb.drift_evaluator.drift_evaluators import BaseDriftEvaluator
from driftdb.drift_evaluator.interface import DriftEvaluatorContext, DriftEvaluation
import pandas as pd

class MyDriftEvaluator(DefaultDriftEvaluator):
    @staticmethod
    def compute_drift_evaluation(data_drift_context: DriftEvaluatorContext)-> DriftEvaluation:
        new_dataframe =  data_drift_context.after
        old_dataframe =  data_drift_context.before
        print("summary", data_drift_context.summary)
        if data_drift_context.summary == None:
            return DriftEvaluation(should_alert= False, message="No summary")
        additions = data_drift_context.summary["added_rows"]
        deletions = data_drift_context.summary["deleted_rows"]
        diff = data_drift_context.summary["modified_patterns"]
        diff_column = diff.columns[0][0]
        diff = diff.apply(pd.to_numeric)  # Convert the values in the diff dataframe to numeric values
        
        print("computing the diff")
        monthly_booking_drift = diff[(diff_column, "other")] - diff[(diff_column, "self")]
        print("monthly_booking_drift", monthly_booking_drift)
        if(monthly_booking_drift > 1000).any():
            return DriftEvaluation(should_alert=True, message= "The monthly booking drift is too high")
        return DriftEvaluation(should_alert= False, message= "Small drift")
    



drift = run_drift_evaluator(gh_client=gh_client, repo_name=repo, commit_sha=commit_sha, drift_evaluator=MyDriftEvaluator())
print("large_drift", drift)

NameError: name 'DefaultDriftEvaluator' is not defined

In [ ]:
import json
import pandas as pd 

coucou = pd.Index(['2023-01-Charlie'])